In [4]:
from PIL import Image
import os
import sys
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F


def convert_image_to_numpy(image_dir):
    image_files = [f for f in os.listdir(image_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
    image_array = []
    for file in image_files:
        img_path = os.path.join(image_dir, file)
        img = Image.open(img_path)
        img = img.convert('RGB')
        img = img.resize((224, 224))
        img_array = np.array(img)
        image_array.append(img_array)
    return np.transpose(np.stack(image_array), (0, 3, 1, 2))      
    

training = convert_image_to_numpy(r"data/training_images")
print(training.shape)

(1001, 3, 224, 224)


In [ ]:
class CNN(nn.Module):
    def __init__(self):
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.f1c1 = nn.Linear(32 * 56 * 56, 128)
        self.f1c2 = nn.Linear(128, 10)
    def forward():
        